In [0]:
#Montagem da tabela delta de cadastro de deputados em formato delta table para camada silver

from pyspark.sql import functions as F
# Caminho do seu arquivo JSON
caminho_json = "dbfs:/Volumes/workspace/deputados/deputados/bronze/cadastro_dep/deputados_*.json"

# Lê o arquivo JSON
df = spark.read.option("multiline", "true").json(caminho_json)
df.printSchema()


df_cad_dep = df.select(
    F.explode("dados").alias("deputado")
).select(
    F.col("deputado.id").alias("id")
    ,F.col("deputado.nome").alias("nome")
    ,F.col("deputado.siglaPartido").alias("sigla_partido")
    ,F.col("deputado.siglaUf").alias("sigla_uf")
    ,F.col("deputado.urlFoto").alias("url_foto")
    #,F.col("deputado.href").alias("url_site")
    ,F.col("deputado.uri").alias("uri")
    ,F.col("deputado.uriPartido").alias("uri_partido")
    ,F.col("deputado.idLegislatura").alias("id_legislatura")
    ,F.col("deputado.email").alias("email")

)

df_cad_dep.show(10)

# Adiciona timestamp de processamento
df_cad_dep = df_cad_dep \
    .withColumn("data_processamento", F.current_timestamp())

# Define o caminho para a Delta Table
delta_table_path = "/Volumes/workspace/deputados/deputados/silver/cad_deputados"

# Cria a Delta Table
df_cad_dep.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .option("delta.constraints.pk_id", "id IS NOT NULL") \
    .save(delta_table_path)

print("✅ Delta Table criada com sucesso!")
print(f" Local: {delta_table_path}")




In [0]:
# Registra como tabela no catalog (para usar com SQL)
df_cad_dep.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .option("delta.constraints.pk_id", "id IS NOT NULL") \
    .saveAsTable("workspace.deputados.silver_cadastro_deputados")

print("✅ Tabela 'despesas_deputados' registrada no catalog!")

In [0]:
%sql

SELECT *
FROM workspace.deputados.silver_cadastro_deputados